# 短期热点币种

需要实时数据，最快的平台是transpose

```sql
select to_token_address, count(*) as tx_counts from ethereum.dex_swaps where timestamp > (now() - INTERVAL '15 minutes') and to_token_address not in ('0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2', '0xdAC17F958D2ee523a2206206994597C13D831ec7', '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48') group by to_token_address order by tx_counts desc;
```
15分钟内的
从结果看有几个地址需要排除（以太坊）
1. 0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2 WETH的地址
2. 0xdAC17F958D2ee523a2206206994597C13D831ec7 USDT
3. 0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48 USDC
4. 0x0000000000000000000000000000000000000000 

修改后的代码
```sql
WITH ExcludedAddresses AS (
    SELECT unnest(ARRAY[
        '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2', 
        '0xdAC17F958D2ee523a2206206994597C13D831ec7', 
        '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48',
        '0x0000000000000000000000000000000000000000'
    ]) AS address
) 
SELECT to_token_address, COUNT(*) as tx_counts 
FROM ethereum.dex_swaps 
WHERE timestamp > (now() - INTERVAL '15 minutes') 
AND to_token_address NOT IN (SELECT address FROM ExcludedAddresses)
GROUP BY to_token_address 
ORDER BY tx_counts DESC;
```

接下来的问题是我如何排除那些女巫账户。如果我们拿着这种模型来交易，很容易就被攻击了。

# 一段时间内的涨跌幅排行

```sql
WITH TokenTime AS (
    SELECT 
        token_address,
        MAX(timestamp) AS end_time,
        MIN(timestamp) AS start_time,
        MAX(price) AS max_price, -- 获取时间段内的最高价
        MIN(price) AS min_price  -- 获取时间段内的最低价
    FROM 
        ethereum.token_prices
    WHERE 
        timestamp > (now() - INTERVAL '15 minutes')
    GROUP BY
        token_address
),
StartPrices AS (
    SELECT 
        tt.token_address,
        tp.price AS start_price
    FROM 
        TokenTime tt
    JOIN 
        ethereum.token_prices tp ON tt.token_address = tp.token_address AND tt.start_time = tp.timestamp
),
EndPrices AS (
    SELECT 
        tt.token_address,
        tp.price AS end_price
    FROM 
        TokenTime tt
    JOIN 
        ethereum.token_prices tp ON tt.token_address = tp.token_address AND tt.end_time = tp.timestamp
)
SELECT 
    sp.token_address,
    sp.start_price,
    ep.end_price,
    tt.max_price,  -- 加入最高价
    tt.min_price,  -- 加入最低价
    (end_price - start_price) / start_price as change_rate
FROM 
    StartPrices sp
JOIN 
    EndPrices ep ON sp.token_address = ep.token_address
JOIN 
    TokenTime tt ON sp.token_address = tt.token_address  -- 连接TokenTime以获取最高价和最低价
ORDER BY 
    change_rate desc;
```

按照短期的涨跌幅排序